In [2]:
import os
import numpy as np
import cv2
import tensorflow as tf
import keras
import mrcnn.config
import mrcnn.utils
#import keras.backend as
from mrcnn.model import MaskRCNN
from pathlib import Path
from twilio.rest import Client


Using TensorFlow backend.


In [3]:
#! pip install keras --upgrade

In [4]:
# Configuration that will be used by the Mask-RCNN library
class MaskRCNNConfig(mrcnn.config.Config):
    NAME = "coco_pretrained_model_config"
    IMAGES_PER_GPU = 1
    GPU_COUNT = 1
    NUM_CLASSES = 1 + 80  # COCO dataset has 80 classes + one background class
    DETECTION_MIN_CONFIDENCE = 0.6

In [5]:
# Filter a list of Mask R-CNN detection results to get only the detected 'bicycle' 3, 'car'4, 'motorcycle'5,'bus'7, 'truck'9 
# coco class names: https://gist.github.com/ageitgey/b143ee809bf08e4927dd59bace44db0d

def get_car_boxes(boxes, class_ids):
    car_boxes = []

    for i, box in enumerate(boxes):
        # If the detected object isn't a bicycle/car/motorcycle/bus/truck, skip it
        if class_ids[i] in [2, 3, 4, 6, 8]:
            car_boxes.append(box)

    return np.array(car_boxes)

In [6]:
# Root directory of the project
ROOT_DIR = Path(".")

In [7]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

In [8]:
# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

In [9]:
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    mrcnn.utils.download_trained_weights(COCO_MODEL_PATH)

In [10]:
# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "Images")

In [11]:
# Video file or camera to process - set this to 0 to use your webcam instead of a video file
VIDEO_SOURCE = "parking.mp4"

In [12]:
# Create a Mask-RCNN model in inference mode
model = MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=MaskRCNNConfig())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [13]:
# Load pre-trained model
model.load_weights(COCO_MODEL_PATH, by_name=True)

In [14]:
# Location of parking spaces
parked_car_boxes = None

In [15]:
# Load the video file we want to run detection on
video_capture = cv2.VideoCapture(VIDEO_SOURCE)

In [16]:
## Loop over each frame of video
while video_capture.isOpened():
    success, frame = video_capture.read()
    if not success:
        break

# Convert the image from BGR color (which OpenCV uses) to RGB color
    rgb_image = frame[:, :, ::-1]
    

In [17]:
# Run the image through the Mask R-CNN model to get results.
results = model.detect([rgb_image], verbose=0)

In [18]:
    # Mask R-CNN assumes we are running detection on multiple images.
    # We only passed in one image to detect, so only grab the first result.
    r = results[0]

    # The r variable will now have the results of detection:
    # - r['rois'] are the bounding box of each detected object
    # - r['class_ids'] are the class id (type) of each detected object
    # - r['scores'] are the confidence scores for each detection
    # - r['masks'] are the object masks for each detected object (which gives you the object outline)

    # Filter the results to only grab the car / truck bounding boxes
    car_boxes = get_car_boxes(r['rois'], r['class_ids'])

    print("Cars found in frame of video:")

    # Draw each box on the frame
    for box in car_boxes:
        print("Car: ", box)

        y1, x1, y2, x2 = box

        # Draw the box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)

    # Show the frame of video on the screen
    cv2.imshow('Video', frame)

    # Hit 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up everything when finished
video_capture.release()
cv2.destroyAllWindows()

SyntaxError: 'break' outside loop (<ipython-input-18-1e07f9734f0a>, line 33)

## The main Program

In [19]:
import os
import numpy as np
import cv2
import mrcnn.config
import mrcnn.utils
from mrcnn.model import MaskRCNN
from pathlib import Path
from twilio.rest import Client

**Configuration that will be used by the Mask-RCNN library**

In [20]:
class MaskRCNNConfig(mrcnn.config.Config):
    NAME = "coco_pretrained_model_config"
    IMAGES_PER_GPU = 1
    GPU_COUNT = 1
    NUM_CLASSES = 1 + 80  # COCO dataset has 80 classes + one background class
    DETECTION_MIN_CONFIDENCE = 0.6

**# Filter a list of Mask R-CNN detection results to get only the detected 'bicycle' 3, 'car'4, 'motorcycle'5,'bus'7, 'truck'9**
#coco class names: https://gist.github.com/ageitgey/b143ee809bf08e4927dd59bace44db0d

In [21]:
def get_car_boxes(boxes, class_ids):
    car_boxes = []

    for i, box in enumerate(boxes):
# If the detected object isn't a bicycle/car/motorcycle/bus/truck, skip it
        if class_ids[i] in [2, 3, 4, 6, 8]:
            car_boxes.append(box)

    return np.array(car_boxes)

**Twilio config**

In [22]:
twilio_account_sid = 'YOUR_TWILIO_SID'
twilio_auth_token = 'YOUR_TWILIO_AUTH_TOKEN'
twilio_phone_number = 'YOUR_TWILIO_SOURCE_PHONE_NUMBER'
destination_phone_number = 'THE_PHONE_NUMBER_TO_TEXT'
client = Client(twilio_account_sid, twilio_auth_token)

**Root directory of the project**

In [23]:
ROOT_DIR = Path(".")

**Directory to save logs and trained model**

In [24]:
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

**Local path to trained weights file**

In [25]:
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

**Download COCO trained weights from Releases if needed**

In [26]:
if not os.path.exists(COCO_MODEL_PATH):
    mrcnn.utils.download_trained_weights(COCO_MODEL_PATH)

**Directory of images to run detection on**

In [27]:
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

**Video file or camera to process - set this to 0 to use your webcam instead of a video file**

In [28]:
VIDEO_SOURCE = "parking.mp4"

**Create a Mask-RCNN model in inference mode**

In [29]:
model = MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=MaskRCNNConfig())

**Load pre-trained model**

In [30]:
model.load_weights(COCO_MODEL_PATH, by_name=True)

**Location of parking spaces**

In [31]:
parked_car_boxes = None

**Load the video file we want to run detection on**

In [32]:
video_capture = cv2.VideoCapture(VIDEO_SOURCE)

**How many frames of video we've seen in a row with a parking space open**

In [33]:
free_space_frames = 0

**Have we sent an SMS alert yet?**

In [34]:
sms_sent = False

**Loop over each frame of video**

In [38]:
while video_capture.isOpened():
    success, frame = video_capture.read()
    print(success)
    if not success:
        break
# Convert the image from BGR color (which OpenCV uses) to RGB color
    rgb_image = frame[:, :, : :-1]

    # Run the image through the Mask R-CNN model to get results.
    results = model.detect([rgb_image], verbose=0)

False


In [39]:
print(rgb_image)

[[[ 96  97 100]
  [ 96  97 100]
  [ 96  97 100]
  ...
  [200 198 194]
  [198 196 192]
  [200 198 194]]

 [[103 104 107]
  [103 104 107]
  [103 104 107]
  ...
  [200 198 194]
  [198 196 192]
  [200 198 194]]

 [[106 111 110]
  [106 111 110]
  [106 111 110]
  ...
  [200 198 194]
  [201 199 195]
  [202 200 196]]

 ...

 [[ 11  29  14]
  [ 13  31  16]
  [ 20  38  23]
  ...
  [157 166 168]
  [157 168 169]
  [158 169 170]]

 [[ 16  36  18]
  [ 19  39  21]
  [ 25  45  27]
  ...
  [153 162 164]
  [152 163 164]
  [153 164 165]]

 [[ 19  39  21]
  [ 26  46  28]
  [ 32  52  34]
  ...
  [123 132 134]
  [123 134 135]
  [127 138 139]]]


In [47]:


    # Mask R-CNN assumes we are running detection on multiple images.
    # We only passed in one image to detect, so only grab the first result.
r = results[0]

    # The r variable will now have the results of detection:
    # - r['rois'] are the bounding box of each detected object
    # - r['class_ids'] are the class id (type) of each detected object
    # - r['scores'] are the confidence scores for each detection
    # - r['masks'] are the object masks for each detected object (which gives you the object outline)

if parked_car_boxes is None:
        # This is the first frame of video - assume all the cars detected are in parking spaces.
        # Save the location of each car as a parking space box and go to the next frame of video.
    parked_car_boxes = get_car_boxes(r['rois'], r['class_ids'])
else:
        # We already know where the parking spaces are. Check if any are currently unoccupied.

        # Get where cars are currently located in the frame
    car_boxes = get_car_boxes(r['rois'], r['class_ids'])

        # See how much those cars overlap with the known parking spaces
    overlaps = mrcnn.utils.compute_overlaps(parked_car_boxes, car_boxes)

        # Assume no spaces are free until we find one that is free
    free_space = False

        # Loop through each known parking space box
    for parking_area, overlap_areas in zip(parked_car_boxes, overlaps):

            # For this parking space, find the max amount it was covered by any
            # car that was detected in our image (doesn't really matter which car)
    max_IoU_overlap = np.max(overlap_areas)

            # Get the top-left and bottom-right coordinates of the parking area
    y1, x1, y2, x2 = parking_area

            # Check if the parking space is occupied by seeing if any car overlaps
            # it by more than 0.15 using IoU
            if max_IoU_overlap < 0.15:
                # Parking space not occupied! Draw a green box around it
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
                # Flag that we have seen at least one open space
                free_space = True
            else:
                # Parking space is still occupied - draw a red box around it
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 1)

            # Write the IoU measurement inside the box
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, f"{max_IoU_overlap:0.2}", (x1 + 6, y2 - 6), font, 0.3, (255, 255, 255))

        # If at least one space was free, start counting frames
        # This is so we don't alert based on one frame of a spot being open.
        # This helps prevent the script triggered on one bad detection.
if free_space:
    free_space_frames += 1
else:
            # If no spots are free, reset the count
        free_space_frames = 0

        # If a space has been free for several frames, we are pretty sure it is really free!
            if free_space_frames > 10:
            # Write SPACE AVAILABLE!! at the top of the screen
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, f"SPACE AVAILABLE!", (10, 150), font, 3.0, (0, 255, 0), 2, cv2.FILLED)

            # If we haven't sent an SMS yet, sent it!
            if not sms_sent:
                print("SENDING SMS!!!")
                message = client.messages.create(
                    body="Parking space open - go go go!",
                    from_=twilio_phone_number,
                    to=destination_phone_number
                )
                sms_sent = True

        # Show the frame of video on the screen
        cv2.imshow('Video', frame)

    # Hit 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up everything when finished
video_capture.release()
cv2.destroyAllWindows()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 83)